# 이탈 예측 성능 끌어올리기

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm

## 데이터 살펴보기

### 유저ID - 이탈 여부

In [3]:
train = pd.read_csv('train_v2_sample.csv')
train

,msno,is_churn
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1
...,...,...
86185,i0SZot2gqC+cRMzwuss0rcuszLxr1jcXuIgIzvwBhqE=,0
86186,bP/tdCEGs0RoW66o1dSLLWDxGMO0NQd9chx/1JhF0co=,0
86187,KzeT/HbgC16Olk2/JwfdoMgNyRBbYaa4o5fJpd/BF9Q=,0
86188,uHEvWYr9kjyjHeU1fAez+wXUYnFhzueK1lSVwW/+5XU=,0


### 유저ID - 유저 정보

* gender, registered_via, city column을 읽을 때 category로 설정

In [5]:
members = pd.read_csv('members_v3_sample.csv', dtype={'gender':'category', 'registered_via': 'category', 'city': 'category'})
members

,msno,city,bd,gender,registered_via,registration_init_time
0,1uvo4CIMRCYGJLiZu029XfvwBqQBiUsMA0ul7+e6kiY=,14,0,NaN,4,20170219
1,20LnYllCVoaG2npEbo9XGABR9r9vTy3PSKxaFkeN6ZQ=,1,0,NaN,7,20100121
2,MvCU75rkklULzbjNSmBcwd0+b2//5javJXXnjdbDL3g=,1,0,NaN,7,20160311
3,h+T+zv9sziRXNmCLSLRBGCu4mPM68/buke6nd+J3FEo=,1,0,NaN,7,20160304
4,Hb0OT/zrvdJE7lYSE5AuPkQ6a/AcJTRxha74GtYNbOA=,17,52,male,9,20090310
...,...,...,...,...,...,...
86185,ZDAbSV5w02oALJjL0lupR9ty4r4WiZTx855YkTpUrqA=,1,0,NaN,7,20120119
86186,DeJ+hdnmqp2+SrnnHljsfitkblX+bDcs+AaCadW9q4Y=,18,30,female,9,20060217
86187,ilT28EbbFan4/E7vCd3ZOudJbNrFL/Qq+l7ziDMIXD4=,13,40,female,9,20051027
86188,zh5KMfqVA//2MhzWgtucjHg6SfBNTLxXw3om61/ML7Y=,5,45,male,9,20100904


In [6]:
members['city']

0        14
1         1
2         1
3         1
4        17
         ..
86185     1
86186    18
86187    13
86188     5
86189    13
Name: city, Length: 86190, dtype: category
Categories (21, object): ['1', '10', '11', '12', ..., '6', '7', '8', '9']

### 유저ID - 거래 내역

In [8]:
transactions = pd.read_csv('transactions_v2.csv', dtype={'payment_method_id': 'category'})
transactions

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0
...,...,...,...,...,...,...,...,...,...
1431004,zwF50wwaJI2TBKWhB42HRBJ6EQK0jgSo1Xmwb9Jq3SU=,32,180,536,536,0,20170215,20170817,0
1431005,zx/h5MzQQmsSat04wSfGpHp6N8aWLLwM1+7OV7ujmPY=,41,30,149,149,1,20170306,20170406,0
1431006,zxvgjIKjy18Fm+cIWUfYKr68z09+ILBxuMW0DnbeUZ8=,41,30,99,99,1,20170308,20170408,0
1431007,zzNhkExbpzmpjp9tXefiCUBtgNLgS+vZE7fFfTRDJVc=,38,30,149,149,0,20170318,20170417,0


## train 데이터와 members 데이터 합치기

In [9]:
train_members = pd.merge(left=train, right=members, how='left', on='msno')
train_members

,msno,is_churn,city,bd,gender,registered_via,registration_init_time
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,0,NaN,7,20140129
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,0,NaN,7,20140515
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1,13,0,NaN,9,20140605
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1,14,49,male,7,20140611
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1,12,21,female,9,20140719
...,...,...,...,...,...,...,...
86185,i0SZot2gqC+cRMzwuss0rcuszLxr1jcXuIgIzvwBhqE=,0,14,37,male,9,20110709
86186,bP/tdCEGs0RoW66o1dSLLWDxGMO0NQd9chx/1JhF0co=,0,14,27,female,9,20110712
86187,KzeT/HbgC16Olk2/JwfdoMgNyRBbYaa4o5fJpd/BF9Q=,0,14,38,female,9,20110806
86188,uHEvWYr9kjyjHeU1fAez+wXUYnFhzueK1lSVwW/+5XU=,0,22,28,male,9,20110812


## transaction 데이터에서 정보를 뽑아낼 준비

In [11]:
merged_transactions = pd.merge(left=train_members, right=transactions, how='left', on='msno')
merged_transactions

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,0,NaN,7,20140129,41,30.0,99.0,99.0,1.0,20170307.0,20170307.0,1.0
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,0,NaN,7,20140515,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1,13,0,NaN,9,20140605,40,30.0,149.0,149.0,1.0,20170311.0,20170314.0,1.0
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1,14,49,male,7,20140611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1,12,21,female,9,20140719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105169,bP/tdCEGs0RoW66o1dSLLWDxGMO0NQd9chx/1JhF0co=,0,14,27,female,9,20110712,40,30.0,149.0,149.0,1.0,20170301.0,20170331.0,0.0
105170,KzeT/HbgC16Olk2/JwfdoMgNyRBbYaa4o5fJpd/BF9Q=,0,14,38,female,9,20110806,40,30.0,149.0,149.0,1.0,20170309.0,20170414.0,0.0
105171,uHEvWYr9kjyjHeU1fAez+wXUYnFhzueK1lSVwW/+5XU=,0,22,28,male,9,20110812,39,30.0,149.0,149.0,1.0,20170331.0,20170512.0,0.0
105172,uHEvWYr9kjyjHeU1fAez+wXUYnFhzueK1lSVwW/+5XU=,0,22,28,male,9,20110812,39,30.0,149.0,149.0,1.0,20170228.0,20170412.0,0.0


## 유저별 거래 통계량 구하기

In [19]:
stats = merged_transactions.groupby('msno').agg({
    'msno': 'count', 'plan_list_price': 'sum', 'actual_amount_paid': 'mean'
}).rename(columns={
    'msno': 'total_order', 'plan_list_price': 'plan_not_worth', 'actual_amount_paid': 'mean_payment_each_transaction'
}).reset_index()
stats

,msno,total_order,plan_not_worth,mean_payment_each_transaction
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1,99.0,99.0
1,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,1,149.0,149.0
2,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,1,149.0,149.0
3,++BW1PJYU5OZi3n3+IDLiU+d1IL1VE/GLx6p64TDs6U=,1,149.0,149.0
4,++Ck01c3EF07Ejek2jfXlKut+sEfg+0ry+A5uWeL9vY=,1,99.0,99.0
...,...,...,...,...
86185,zzls+AKrL4wxULaWF3BU5QjFpWe4mqN/EAvZFfxR3us=,1,149.0,149.0
86186,zzm2UvJnzuTRkXaiaZHtbJwPG9jZQZkZxG0n4PYDTvw=,1,99.0,99.0
86187,zzompfSaMamqvjyCMYvgUBwYrxh8fHE40z3f73CQoak=,1,149.0,149.0
86188,zztb7cCNxnkeZ33rHQ0Z3cnshdE8gn/nwMTiNP49b4o=,2,298.0,149.0


## 유저 정보에 거래 이력 통계 정보를 합치기

In [20]:
df = pd.merge(left=train_members, right=stats, how='left', on='msno')
df

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,total_order,plan_not_worth,mean_payment_each_transaction
0,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,1,0,NaN,7,20140129,1,99.0,99.0
1,WkF/FvlzpBLFoa+Hm3hagOfme2pHLE2y+RJ3eGcLT0k=,1,1,0,NaN,7,20140515,1,0.0,NaN
2,w2O0vchmncaBEQLoC5Pn7qRoSPiJ6X2x5O+woVFDhnY=,1,13,0,NaN,9,20140605,1,149.0,149.0
3,SlkPQrh6oIqsdpW+Sl2adYlJNEwWSQsIYycM11dSUQg=,1,14,49,male,7,20140611,1,0.0,NaN
4,Z9Y6f2hXZpPIvbLgHHgUncs7Tb+qmKgGNSwwVAiMlm0=,1,12,21,female,9,20140719,1,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...
86185,i0SZot2gqC+cRMzwuss0rcuszLxr1jcXuIgIzvwBhqE=,0,14,37,male,9,20110709,1,149.0,149.0
86186,bP/tdCEGs0RoW66o1dSLLWDxGMO0NQd9chx/1JhF0co=,0,14,27,female,9,20110712,1,149.0,149.0
86187,KzeT/HbgC16Olk2/JwfdoMgNyRBbYaa4o5fJpd/BF9Q=,0,14,38,female,9,20110806,1,149.0,149.0
86188,uHEvWYr9kjyjHeU1fAez+wXUYnFhzueK1lSVwW/+5XU=,0,22,28,male,9,20110812,2,298.0,149.0


## 아웃라이어 제거

In [21]:
df['bd'].min(), df['bd'].max()

(-3152, 1035)

In [24]:
df['bd'].clip(0, 100, inplace=True)
df['bd'].fillna(0, inplace=True)
df['bd'].min(), df['bd'].max()

(0, 100)

## 필요없는 column, 결측값 제거

In [25]:
df.drop(columns='msno', inplace=True)
df.dropna(inplace=True)

## 더미화

In [28]:
df_dummies = pd.get_dummies(df)
df_dummies

,is_churn,bd,registration_init_time,total_order,plan_not_worth,mean_payment_each_transaction,city_1,city_10,city_11,city_12,...,city_7,city_8,city_9,gender_female,gender_male,registered_via_13,registered_via_3,registered_via_4,registered_via_7,registered_via_9
5,1,31,20140828,1,149.0,149.0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
6,1,19,20141112,2,2400.0,1200.0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
11,1,22,20150920,1,149.0,149.0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
12,1,33,20150922,1,536.0,536.0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
14,1,19,20160204,3,447.0,149.0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86183,0,56,20110624,1,149.0,149.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
86185,0,37,20110709,1,149.0,149.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
86186,0,27,20110712,1,149.0,149.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
86187,0,38,20110806,1,149.0,149.0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [29]:
X = df_dummies.drop(columns='is_churn')
y = df_dummies['is_churn'].values

In [30]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=0)

# 랜덤 포레스트(Random Forest)

## DecisionTreeClassifier

### 모델 초기화 및 학습하기

In [32]:
from sklearn.tree import DecisionTreeClassifier

In [33]:
tree_model = DecisionTreeClassifier(random_state=0, max_depth=2)
tree_model.fit(train_X, train_y)

DecisionTreeClassifier(max_depth=2, random_state=0)

### 모델 평가하기(정확도)

In [35]:
from sklearn import metrics
prediction_test = tree_model.predict(test_X)
print(metrics.f1_score(test_y, prediction_test))

0.6349717969379532


## Random Forest

### 모델 초기화 및 학습하기

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(train_X, train_y)

RandomForestClassifier()

### 모델 평가하기(정확도)

In [39]:
from sklearn import metrics
prediction_test = rf_model.predict(test_X)
print(metrics.f1_score(test_y, prediction_test))

0.6661687826736371


# 에이다부스트(AdaBoost)

## AdaBoostClassifier

### 모델 초기화 및 학습하기

In [40]:
from sklearn.ensemble import AdaBoostClassifier

In [43]:
ada_model = AdaBoostClassifier(n_estimators=100)
ada_model.fit(train_X, train_y)

AdaBoostClassifier(n_estimators=100)

### 모델 평가하기(정확도)

In [44]:
from sklearn import metrics
prediction_test = ada_model.predict(test_X)
print(metrics.f1_score(test_y, prediction_test))

0.6518987341772151
